<a href="https://colab.research.google.com/github/redinbluesky/nlp-with-transformers/blob/main/03-트랜스포머 파헤치기.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  텍스트 분류 목차
* [Chapter 0 개요](#chapter0)
* [Chapter 1 데이터셋](#chapter1)

## Chapter 0 개요 <a class="anchor" id="chapter0"></a>
1. 이 장에서는 트랜스포머 모델의 주요 구성 요소와 이를 파이토치로 구현하는 방법을 살펴본다.
   - 텐서플로우로 동일한 작업을 수행하는 방법도 알아본다.

2. 어텐션 메커니즘을 만드는데 초점을 맞춘 다음, 트랜스포머 인코더를 구현하기 위한 필요 요소를 추가한다.

3. 문제를 해결하기 위해서 트랜스포머를 사용하고 모델을 미세 튜닝하는데 트랜스포머 아키텍처의 기술적인 측면을 이해할 필요는 없다.
   - 하지만 기술적인 측면을 알면 트랜스포먼의 한계점을 파악하거나 새로운 도메일에 적용할 때 도움이 된다.
